In [1]:
import numpy as np
import sympy as sp

def pfin_achain_bdy(mflags):
    def flags_from_max_flags(mflags):
        def powerset(iterable):
            from itertools import chain, combinations
            "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
            s = list(iterable)
            return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))    
        fl = mflags
        all_flags = mflags
        for flag in mflags:
            contained_flags = powerset(flag)
            all_flags += contained_flags
            all_flags = sorted(set(all_flags),key=lambda x: (-len(x),x))
        return all_flags    
    def boundary_from_flags(flags):
        def _ker(A:sp.Matrix):
            k = A.nullspace()
            return sp.Matrix(np.hstack(k))
        def _im(A:sp.Matrix):
            i = A.columnspace()
            return sp.Matrix(np.hstack(i))
        def _dec(A):
            return _ker(A), _im(A)
        flags = sorted(set(flags),key=lambda x: (-len(x),x))
        A = np.zeros((len(flags),len(flags)),dtype=np.int8)
        flag_dict = dict(enumerate(flags))
        for i,flag in enumerate(flags):
            for j in range(len(flag)):
                sign = (-1)**j; bdj = tuple(sorted(list(flag[:j])+list(flag[j+1:])))
                t_ix = flags.index(bdj)
                A[t_ix,i] = sign
        A = sp.Matrix(A)
        ker,im = _dec(A)
        return A,ker,im,flag_dict
    res = boundary_from_flags(flags_from_max_flags(mflags))
    assert np.array(res[0]*res[1]).max() == 0, "kernel not a kernel"
    assert np.array(res[0]*res[2]).max() == 0, "boundaries not subset of kernel"
    A,ker,im,flag_dict = res
    dev_b = ker.solve(im) # erfuellt Z*dev_b = B .. i.e. ist matrix rep von im ( i: B -> Z ) sub Z
    #assert that it actually is a monomorphism as it should be
    assert len(dev_b.nullspace()) == 0, "dev_b not injective, sth wrong"
    assert dev_b.rank() == min(dev_b.shape), "dev_b not of full rank, sth wrong"    
    #which means want to calc 0->B->Z->Z/B = H->0, i.e. coker dev_b,
    #i.e. Z/im dev_b, so we need to diagonalise dev_b, so the quotient
    # can just be read off
    
    return (A,ker,dev_b,flag_dict)
    
rp2_antichain = [(0,1,5),(1,2,5),(2,5,6),(0,2,6),(0,1,6),(1,4,6),(1,2,4),(0,2,4),(0,4,5),(4,5,6)]
D,Z,B,simplex_dict = pfin_achain_bdy(rp2_antichain)

In [35]:
M = sp.Matrix(np.random.randint(-3,3,size=(8,10)))
M

Matrix([
[-3, -3, -3,  1,  2, -3, -2,  0, -1, -3],
[-1,  1,  0,  0, -3,  2, -1, -3,  1, -1],
[ 0, -2,  2, -1, -2, -1, -1, -2,  2,  0],
[-1,  2, -2,  1,  0,  1,  2, -1,  2,  2],
[ 1,  2, -2,  0, -2, -2,  1,  0, -1, -3],
[ 2,  0, -2, -3,  0, -2,  2,  2,  0, -1],
[ 0, -2,  0,  1, -3, -3,  0, -1,  2,  0],
[ 1,  0, -3,  1, -3,  2, -1,  1,  1, -2]])

In [38]:
P,L,dia,U=M.LUdecompositionFF()

In [44]:
P**(-1)*L*dia**(-1)*U

Matrix([
[-3, -3, -3,  1,  2, -3, -2,  0, -1, -3],
[-1,  1,  0,  0, -3,  2, -1, -3,  1, -1],
[ 0, -2,  2, -1, -2, -1, -1, -2,  2,  0],
[-1,  2, -2,  1,  0,  1,  2, -1,  2,  2],
[ 1,  2, -2,  0, -2, -2,  1,  0, -1, -3],
[ 2,  0, -2, -3,  0, -2,  2,  2,  0, -1],
[ 0, -2,  0,  1, -3, -3,  0, -1,  2,  0],
[ 1,  0, -3,  1, -3,  2, -1,  1,  1, -2]])